In [26]:
pip install -U skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 155 kB 27.9 MB/s 


In [37]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.datasets import make_classification

In [42]:
X, y = make_classification(2000, 10, random_state=0)
X, y = X.astype(np.float32), y.astype(np.int64)

In [43]:
X.shape

(2000, 10)

In [44]:
y.shape

(2000,)

In [45]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=30,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(10, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

In [46]:
from skorch import NeuralNetClassifier

In [47]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

In [48]:
net.get_params()

{'module': __main__.ClassifierModule,
 'criterion': torch.nn.modules.loss.NLLLoss,
 'optimizer': torch.optim.sgd.SGD,
 'lr': 0.1,
 'max_epochs': 20,
 'batch_size': 128,
 'iterator_train': torch.utils.data.dataloader.DataLoader,
 'iterator_valid': torch.utils.data.dataloader.DataLoader,
 'dataset': skorch.dataset.Dataset,
 'train_split': <skorch.dataset.ValidSplit object at 0x7f54811fcfd0>,
 'callbacks': None,
 'predict_nonlinearity': 'auto',
 'warm_start': False,
 'verbose': 1,
 'device': 'cpu',
 '_kwargs': {},
 'classes': None,
 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer at 0x7f54811029d0>,
 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring at 0x7f5481022810>,
 'callbacks__train_loss__name': 'train_loss',
 'callbacks__train_loss__lower_is_better': True,
 'callbacks__train_loss__on_train': True,
 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring at 0x7f5481017d90>,
 'callbacks__valid_loss__name': 'valid_loss',
 'callbacks__val

In [54]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7076       0.5325        0.6674  0.0255
      2        0.6532       0.8050        0.5975  0.0217
      3        0.5660       0.9675        0.4638  0.0206
      4        0.4265       0.9800        0.2979  0.0189
      5        0.2913       0.9875        0.1789  0.0207
      6        0.2128       0.9925        0.1150  0.0208
      7        0.1689       0.9900        0.0825  0.0207
      8        0.1496       0.9900        0.0644  0.0195
      9        0.1183       0.9900        0.0545  0.0197
     10        0.1218       0.9900        0.0490  0.0198
     11        0.1240       0.9900        0.0464  0.0203
     12        0.1090       0.9900        0.0428  0.0213
     13        0.1050       0.9900        0.0410  0.0215
     14        0.1067       0.9875        0.0399  0.0214
     15        0.1072       0.9900        0.0392  0.0211
     16        0.0958       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=10, out_features=30, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=30, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [55]:
list(net.get_params())

['module',
 'criterion',
 'optimizer',
 'lr',
 'max_epochs',
 'batch_size',
 'iterator_train',
 'iterator_valid',
 'dataset',
 'train_split',
 'callbacks',
 'predict_nonlinearity',
 'warm_start',
 'verbose',
 'device',
 '_kwargs',
 'classes',
 'callbacks__epoch_timer',
 'callbacks__train_loss',
 'callbacks__train_loss__name',
 'callbacks__train_loss__lower_is_better',
 'callbacks__train_loss__on_train',
 'callbacks__valid_loss',
 'callbacks__valid_loss__name',
 'callbacks__valid_loss__lower_is_better',
 'callbacks__valid_loss__on_train',
 'callbacks__valid_acc',
 'callbacks__valid_acc__scoring',
 'callbacks__valid_acc__lower_is_better',
 'callbacks__valid_acc__on_train',
 'callbacks__valid_acc__name',
 'callbacks__valid_acc__target_extractor',
 'callbacks__valid_acc__use_caching',
 'callbacks__print_log',
 'callbacks__print_log__keys_ignored',
 'callbacks__print_log__sink',
 'callbacks__print_log__tablefmt',
 'callbacks__print_log__floatfmt',
 'callbacks__print_log__stralign']

In [56]:
y_pred = net.predict(X[:5])
y_pred

array([1, 0, 0, 1, 1])

In [57]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[7.7738642e-04, 9.9922264e-01],
       [9.9628782e-01, 3.7122301e-03],
       [9.9648917e-01, 3.5108225e-03],
       [3.2411060e-01, 6.7588937e-01],
       [4.5940662e-03, 9.9540591e-01]], dtype=float32)

In [58]:
from sklearn.datasets import make_regression

In [59]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

In [60]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000, 1), -6.4901485, 6.154505)

In [61]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [62]:
from skorch import NeuralNetRegressor

In [63]:
net_regr = NeuralNetRegressor(
    RegressorModule,
    max_epochs=20,
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

In [64]:
net_regr.fit(X_regr, y_regr)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.3247        3.0078  0.0170
      2        1.7262        0.6808  0.0123
      3        0.6510        0.2147  0.0115
      4        0.1811        0.2132  0.0118
      5        0.1906        0.1127  0.0108
      6        0.1143        0.3361  0.0204
      7        0.3835        0.0899  0.0113
      8        0.0845        0.1574  0.0117
      9        0.1099        0.0486  0.0130
     10        0.0485        0.0974  0.0128
     11        0.0907        0.0447  0.0108
     12        0.0481        0.0947  0.0129
     13        0.0881        0.0322  0.0128
     14        0.0323        0.0599  0.0117
     15        0.0461        0.0180  0.0115
     16        0.0161        0.0328  0.0123
     17        0.0231        0.0125  0.0123
     18        0.0098        0.0208  0.0123
     19        0.0143        0.0102  0.0112
     20        0.0074        0.0153  0.0121


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

In [65]:
y_pred = net_regr.predict(X_regr[:5])
y_pred

array([[ 0.7368696 ],
       [-1.2884711 ],
       [-0.51758516],
       [-0.11890286],
       [-0.61254007]], dtype=float32)

In [66]:
import pickle

In [67]:
file_name = '/tmp/mymodel.pkl'

In [68]:
with open(file_name, 'wb') as f:
    pickle.dump(net, f)

In [69]:
with open(file_name, 'rb') as f:
    new_net = pickle.load(f)

In [70]:
net.save_params(f_params=file_name)  # a file handler also works

In [71]:
# first initialize the model
new_net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
).initialize()

In [72]:
new_net.load_params(file_name)

In [73]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [74]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

In [75]:
pipe.fit(X, y)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6925       0.5250        0.6640  0.0355
      2        0.6361       0.9075        0.5834  0.0354
      3        0.5447       0.9550        0.4427  0.0340
      4        0.4197       0.9675        0.2898  0.0207
      5        0.3019       0.9775        0.1798  0.0214
      6        0.2282       0.9825        0.1206  0.0217
      7        0.1790       0.9875        0.0869  0.0207
      8        0.1550       0.9875        0.0697  0.0208
      9        0.1473       0.9875        0.0594  0.0196
     10        0.1249       0.9875        0.0525  0.0200
     11        0.1294       0.9900        0.0482  0.0213
     12        0.1194       0.9925        0.0446  0.0220
     13        0.1192       0.9950        0.0428  0.0291
     14        0.1035       0.9925        0.0406  0.0215
     15   

Pipeline(steps=[('scale', StandardScaler()),
                ('net',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=10, out_features=30, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=30, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
))])

In [76]:
y_proba = pipe.predict_proba(X[:5])
y_proba

array([[0.00224374, 0.9977563 ],
       [0.9986193 , 0.00138069],
       [0.99899906, 0.00100095],
       [0.30393705, 0.6960629 ],
       [0.00816792, 0.9918321 ]], dtype=float32)

In [77]:
from skorch.callbacks import EpochScoring

In [78]:
auc = EpochScoring(scoring='roc_auc', lower_is_better=False)

In [79]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)

In [80]:
net.fit(X, y)

  epoch    roc_auc    train_loss    valid_acc    valid_loss     dur
-------  ---------  ------------  -----------  ------------  ------
      1     0.9544        0.6614       0.8900        0.6294  0.0193
      2     0.9845        0.5875       0.9625        0.5233  0.0208
      3     0.9899        0.4798       0.9800        0.3647  0.0198
      4     0.9945        0.3600       0.9825        0.2302  0.0208
      5     0.9972        0.2682       0.9850        0.1451  0.0200
      6     0.9975        0.2087       0.9850        0.1002  0.0187
      7     0.9978        0.1869       0.9850        0.0762  0.0191
      8     0.9979        0.1699       0.9850        0.0640  0.0222
      9     0.9980        0.1430       0.9875        0.0567  0.0200
     10     0.9981        0.1338       0.9875        0.0500  0.0201
     11     0.9981        0.1214       0.9875        0.0464  0.0355
     12     0.9981        0.1167       0.9900        0.0442  0.0346
     13     0.9982        0.1072       0.9875   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=10, out_features=30, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=30, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [81]:
print(', '.join(net.prefixes_))

iterator_train, iterator_valid, callbacks, dataset, module, criterion, optimizer


In [82]:
from sklearn.model_selection import GridSearchCV

In [83]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    optimizer__momentum=0.9,
    verbose=0,
    train_split=False,
)

In [84]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0, 0.5],
    'optimizer__nesterov': [False, True],
}

In [85]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

In [86]:
gs.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False; total time=   0.4s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False; total time=   0.3s
[CV] END lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False; total time=   0.

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
),
             param_grid={'lr': [0.05, 0.1], 'module__dropout': [0, 0.5],
                         'module__num_units': [10, 20],
                         'optimizer__nesterov': [False, True]},
             refit=False, scoring='accuracy', verbose=2)

In [87]:
print(gs.best_score_, gs.best_params_)

0.988499744121933 {'lr': 0.1, 'module__dropout': 0.5, 'module__num_units': 20, 'optimizer__nesterov': False}


In [88]:
net

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
)

In [ ]:
#conda install captum -c pytorch
#pip install captum

In [95]:
pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 29.2 MB/s 


In [134]:
# Initial imports
import numpy as np

import torch

from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import pandas as pd

In [135]:
dataset_path = "https://raw.githubusercontent.com/pradmishra1/PublicDatasets/main/titanic.csv"

In [136]:
titanic_data = pd.read_csv(dataset_path)

In [137]:
titanic_data.columns

Index(['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [138]:
del titanic_data['Unnamed: 0']

In [139]:
del titanic_data['PassengerId']

In [140]:
titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [141]:
titanic_data = pd.concat([titanic_data,
                          pd.get_dummies(titanic_data['Sex']),
                          pd.get_dummies(titanic_data['Embarked'],prefix="embark"),
                          pd.get_dummies(titanic_data['Pclass'],prefix="class")], axis=1)
titanic_data["Age"] = titanic_data["Age"].fillna(titanic_data["Age"].mean())
titanic_data["Fare"] = titanic_data["Fare"].fillna(titanic_data["Fare"].mean())
titanic_data = titanic_data.drop(['Name','Ticket','Cabin','Sex','Embarked','Pclass'], axis=1)

In [142]:
# Set random seed for reproducibility.
np.random.seed(707)

# Convert features and labels to numpy arrays.
labels = titanic_data["Survived"].to_numpy()
titanic_data = titanic_data.drop(['Survived'], axis=1)
feature_names = list(titanic_data.columns)
data = titanic_data.to_numpy()

# Separate training and test sets using 
train_indices = np.random.choice(len(labels), int(0.7*len(labels)), replace=False)
test_indices = list(set(range(len(labels))) - set(train_indices))
train_features = data[train_indices]
train_labels = labels[train_indices]
test_features = data[test_indices]
test_labels = labels[test_indices]

In [143]:
train_features.shape

(623, 12)

In [145]:
import torch
import torch.nn as nn
torch.manual_seed(1)  # Set seed for reproducibility.
class TitanicSimpleNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(12, 12)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(12, 8)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(8, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lin1_out = self.linear1(x)
        sigmoid_out1 = self.sigmoid1(lin1_out)
        sigmoid_out2 = self.sigmoid2(self.linear2(sigmoid_out1))
        return self.softmax(self.linear3(sigmoid_out2))

In [146]:
net = TitanicSimpleNNModel()
criterion = nn.CrossEntropyLoss()
num_epochs = 200

optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
input_tensor = torch.from_numpy(train_features).type(torch.FloatTensor)
label_tensor = torch.from_numpy(train_labels)


In [147]:
for epoch in range(num_epochs):
  output = net(input_tensor)
  loss = criterion(output, label_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if epoch % 20 == 0:
        print ('Epoch {}/{} => Loss: {:.2f}'.format(epoch+1, num_epochs, loss.item()))
torch.save(net.state_dict(), '/model.pt')

Epoch 1/200 => Loss: 0.70
Epoch 21/200 => Loss: 0.55
Epoch 41/200 => Loss: 0.50
Epoch 61/200 => Loss: 0.49
Epoch 81/200 => Loss: 0.48
Epoch 101/200 => Loss: 0.49
Epoch 121/200 => Loss: 0.47
Epoch 141/200 => Loss: 0.47
Epoch 161/200 => Loss: 0.47
Epoch 181/200 => Loss: 0.47


In [148]:
out_probs = net(input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Train Accuracy:", sum(out_classes == train_labels) / len(train_labels))

Train Accuracy: 0.8523274478330658


In [149]:
test_input_tensor = torch.from_numpy(test_features).type(torch.FloatTensor)
out_probs = net(test_input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Test Accuracy:", sum(out_classes == test_labels) / len(test_labels))

Test Accuracy: 0.832089552238806


In [150]:
ig = IntegratedGradients(net)

In [151]:
test_input_tensor.requires_grad_()
attr, delta = ig.attribute(test_input_tensor,target=1, return_convergence_delta=True)
attr = attr.detach().numpy()

In [152]:
np.round(attr,2)

array([[-0.7 ,  0.09, -0.  , ...,  0.  ,  0.  , -0.33],
       [-2.78, -0.  , -0.  , ...,  0.  ,  0.  , -1.82],
       [-0.65,  0.  , -0.  , ...,  0.  ,  0.  , -0.31],
       ...,
       [-0.47, -0.  , -0.  , ...,  0.71,  0.  , -0.  ],
       [-0.1 , -0.  , -0.  , ...,  0.  ,  0.  , -0.1 ],
       [-0.7 ,  0.  , -0.  , ...,  0.  ,  0.  , -0.28]])

In [153]:
importances = np.mean(attr, axis=0)

In [154]:
for i in range(len(feature_names)):
        print(feature_names[i], ": ", '%.3f'%(importances[i]))

Age :  -0.574
SibSp :  -0.010
Parch :  -0.026
Fare :  0.278
female :  0.101
male :  -0.460
embark_C :  0.042
embark_Q :  0.005
embark_S :  -0.021
class_1 :  0.067
class_2 :  0.090
class_3 :  -0.144


In [155]:
cond = LayerConductance(net, net.sigmoid1)

In [156]:
cond_vals = cond.attribute(test_input_tensor,target=1)
cond_vals = cond_vals.detach().numpy()

In [158]:
Average_Neuron_Importances = np.mean(cond_vals, axis=0)
Average_Neuron_Importances

array([ 0.03051018, -0.23244175,  0.04743345,  0.02102091, -0.08071412,
       -0.09040915, -0.13398956, -0.04666219,  0.03577907, -0.07206058,
       -0.15658873,  0.03491106], dtype=float32)

In [159]:
neuron_cond = NeuronConductance(net, net.sigmoid1)

In [160]:
neuron_cond_vals_10 = neuron_cond.attribute(test_input_tensor, neuron_selector=10, target=1)

In [161]:
neuron_cond_vals_0 = neuron_cond.attribute(test_input_tensor, neuron_selector=0, target=1)

In [165]:
# Average Feature Importances for Neuron 0
nn0 = neuron_cond_vals_0.mean(dim=0).detach().numpy()
np.round(nn0,3)

array([ 0.008,  0.   ,  0.   ,  0.028,  0.   , -0.004, -0.   ,  0.   ,
       -0.001, -0.   ,  0.   , -0.   ], dtype=float32)